In [83]:
import sqlite3
import numpy as np
import pandas as pd

## Setup dbase in RAM

In [84]:
con = sqlite3.connect(':memory:')
cur = con.cursor()

In [85]:
sql_dump = open(r"db_sql\db_dmp.sql","r")
sql_statements = (sql_dump
                    .read()
                    .split(";"))
sql_dump.close()
                
for st in sql_statements:
    try:
        cur.execute(st)
    except sqlite3.Error as e:
        print(e)              

## Check DB (get list of tables)

In [86]:
pd.read_sql_query(
    """ SELECT name
        FROM sqlite_master 
        WHERE type ='table'
        AND name NOT LIKE 'sqlite_%'; """,
        con)

,name
0,medicine
1,diagnosis
2,job
3,workers
4,contacts


## Run queries in SQL
a. Напишите запрос,который выводит название позиций(без повторений), хотя бы один представитель которой получает зарплату в промежутке 400-600

In [87]:
pd.read_sql_query(
    """ SELECT DISTINCT(POSITION)
        FROM job
        WHERE SALARY BETWEEN 400 AND 600; """,
        con)

,POSITION
0,C# DEVELOPER
1,AUTOMATION QA ENGINEER
2,JAVA DEVELOPER


b. Напишите запрос, который выполняет вывод списка офисов, расположенных в SEBEWAING и ELECTRON, количество рабочих и средней зарплаты рабочих в каждом офисе

In [88]:
pd.read_sql_query(
    """ SELECT OFFICE, count(WORKER_ID), avg(SALARY) as MEAN_SALARY
        FROM job
        WHERE OFFICE_CITY IN ("SEBEWAING", "ELECTRON")
        GROUP BY OFFICE; """,
        con)

,OFFICE,count(WORKER_ID),MEAN_SALARY
0,TIME IS MONEY,7,725.958571


c. Напишите запрос, который выводит название профессий (без повторений), все представители которой получают зарплату в промежутке 400-700

In [89]:
pd.read_sql_query(
    """ SELECT DISTINCT(POSITION)
        FROM job
        WHERE POSITION NOT IN
            (SELECT POSITION
            FROM job
            WHERE SALARY NOT BETWEEN 400 AND 700); """,
        con)

,POSITION
0,AUTOMATION QA ENGINEER
1,JAVA DEVELOPER


d. Напишите запрос, который выводит среднюю зарплату работников, которые старше 30 лет, группируя по полу и возрасту, но при выводе увеличивающий данные о величине зарплаты на 20%

In [90]:
pd.read_sql_query(
    """ SELECT  substr(date(),1,4) - 19||substr(DOB,8,2) AS AGE,
                GENDER,
                AVG(SALARY) * 1.2 AS AVG_SALARY_UP
        FROM workers t1
        INNER JOIN job t2
        ON t1.WORKER_ID = t2.WORKER_ID
        WHERE AGE > 30
        GROUP BY AGE, GENDER; """,
        con)

,AGE,GENDER,AVG_SALARY_UP
0,34,F,1052.370
1,34,M,691.544
2,35,F,1175.004
3,39,F,665.244


e. Напишите запрос, который выведет накопленную зарплату по месяцам для Николы Теслы начиная с его приему на работу до сегодняшнего дня.

In [91]:
pd.read_sql_query("""
WITH RECURSIVE dates AS
  (SELECT min(CASE substr(FROM_DATE, 4, 3)
                  WHEN 'Jan' THEN 20||substr(FROM_DATE, 8, 2)||'-01'
                  WHEN 'Feb' THEN 20||substr(FROM_DATE, 8, 2)||'-02'
                  WHEN 'Mar' THEN 20||substr(FROM_DATE, 8, 2)||'-03'
                  WHEN 'Apr' THEN 20||substr(FROM_DATE, 8, 2)||'-04'
                  WHEN 'May' THEN 20||substr(FROM_DATE, 8, 2)||'-05'
                  WHEN 'Jun' THEN 20||substr(FROM_DATE, 8, 2)||'-06'
                  WHEN 'Jul' THEN 20||substr(FROM_DATE, 8, 2)||'-07'
                  WHEN 'Aug' THEN 20||substr(FROM_DATE, 8, 2)||'-08'
                  WHEN 'Sep' THEN 20||substr(FROM_DATE, 8, 2)||'-09'
                  WHEN 'Oct' THEN 20||substr(FROM_DATE, 8, 2)||'-10'
                  WHEN 'Nov' THEN 20||substr(FROM_DATE, 8, 2)||'-11'
                  WHEN 'Dec' THEN 20||substr(FROM_DATE, 8, 2)||'-12'
              END) AS year_month
   FROM job
   UNION ALL SELECT strftime('%Y-%m', date(year_month||'-01', '+1 month'))
   FROM dates
   WHERE year_month < date('now', '-1 month'))
SELECT year_month,
       SUM(SALARY) OVER (ROWS UNBOUNDED PRECEDING) AS CUM_SALARY
FROM
  (SELECT year_month,
          SALARY
   FROM dates
   INNER JOIN
     (SELECT SALARY,
             CASE substr(FROM_DATE, 4, 3)
                 WHEN 'Jan' THEN 20||substr(FROM_DATE, 8, 2)||'-01'
                 WHEN 'Feb' THEN 20||substr(FROM_DATE, 8, 2)||'-02'
                 WHEN 'Mar' THEN 20||substr(FROM_DATE, 8, 2)||'-03'
                 WHEN 'Apr' THEN 20||substr(FROM_DATE, 8, 2)||'-04'
                 WHEN 'May' THEN 20||substr(FROM_DATE, 8, 2)||'-05'
                 WHEN 'Jun' THEN 20||substr(FROM_DATE, 8, 2)||'-06'
                 WHEN 'Jul' THEN 20||substr(FROM_DATE, 8, 2)||'-07'
                 WHEN 'Aug' THEN 20||substr(FROM_DATE, 8, 2)||'-08'
                 WHEN 'Sep' THEN 20||substr(FROM_DATE, 8, 2)||'-09'
                 WHEN 'Oct' THEN 20||substr(FROM_DATE, 8, 2)||'-10'
                 WHEN 'Nov' THEN 20||substr(FROM_DATE, 8, 2)||'-11'
                 WHEN 'Dec' THEN 20||substr(FROM_DATE, 8, 2)||'-12'
             END AS start_date
      FROM job t1
      INNER JOIN workers t2 ON t1.WORKER_ID = t2.WORKER_ID
      WHERE FIRST_NAME = 'NIKOLA'
        AND LAST_NAME = 'TESLA') t3 ON t3.start_date < dates.year_month);
        """, con)

,year_month,CUM_SALARY
0,2009-07,621.44
1,2009-08,1242.88
2,2009-09,1864.32
3,2009-10,2485.76
4,2009-11,3107.20
...,...,...
126,2020-01,78922.88
127,2020-02,79544.32
128,2020-03,80165.76
129,2020-04,80787.20


f. Напишите запрос, который выведет накопленную зарплату по месяцам, разбитую по позициям, за 2014 год

In [92]:
pd.read_sql_query("""
WITH RECURSIVE dates AS
  (SELECT '2014-01' AS year_month
   UNION ALL SELECT strftime('%Y-%m', date(year_month||'-01', '+1 month'))
   FROM dates
   WHERE year_month < date('2014-12-01', '-1 month'))
SELECT POSITION,
       SUM(CASE
               WHEN year_month = '2014-01' THEN CUM_SALARY
           END) AS '2014-01',
       SUM(CASE
               WHEN year_month = '2014-02' THEN CUM_SALARY
           END) AS '2014-02',
       SUM(CASE
               WHEN year_month = '2014-03' THEN CUM_SALARY
           END) AS '2014-03',
       SUM(CASE
               WHEN year_month = '2014-04' THEN CUM_SALARY
           END) AS '2014-04',
       SUM(CASE
               WHEN year_month = '2014-05' THEN CUM_SALARY
           END) AS '2014-05',
       SUM(CASE
               WHEN year_month = '2014-06' THEN CUM_SALARY
           END) AS '2014-06',
       SUM(CASE
               WHEN year_month = '2014-07' THEN CUM_SALARY
           END) AS '2014-07',
       SUM(CASE
               WHEN year_month = '2014-08' THEN CUM_SALARY
           END) AS '2014-08',
       SUM(CASE
               WHEN year_month = '2014-09' THEN CUM_SALARY
           END) AS '2014-09',
       SUM(CASE
               WHEN year_month = '2014-10' THEN CUM_SALARY
           END) AS '2014-10',
       SUM(CASE
               WHEN year_month = '2014-11' THEN CUM_SALARY
           END) AS '2014-11',
       SUM(CASE
               WHEN year_month = '2014-12' THEN CUM_SALARY
           END) AS '2014-12'
FROM
  (SELECT year_month,
          POSITION,
          SUM(SUM_SALARY) OVER (PARTITION BY POSITION
                                ORDER BY year_month) AS CUM_SALARY
   FROM
     (SELECT year_month,
             POSITION,
             SUM_SALARY
      FROM dates
      INNER JOIN
        (SELECT POSITION,
                SUM(SALARY) AS SUM_SALARY,
                '2014-01' AS mon
         FROM job
         GROUP BY POSITION) t1 ON dates.year_month >= t1.mon))
GROUP BY POSITION;
      """, con)

,POSITION,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12
0,AUTOMATION QA ENGINEER,2849.37,5698.74,8548.11,11397.48,14246.85,17096.22,19945.59,22794.96,25644.33,28493.7,31343.07,34192.44
1,AUTOMATION QA TEAMLEAD,979.17,1958.34,2937.51,3916.68,4895.85,5875.02,6854.19,7833.36,8812.53,9791.7,10770.87,11750.04
2,C# DEVELOPER,2315.91,4631.82,6947.73,9263.64,11579.55,13895.46,16211.37,18527.28,20843.19,23159.1,25475.01,27790.92
3,C# DEVELOPER TEAMLEAD,1021.77,2043.54,3065.31,4087.08,5108.85,6130.62,7152.39,8174.16,9195.93,10217.7,11239.47,12261.24
4,JAVA DEVELOPER,554.37,1108.74,1663.11,2217.48,2771.85,3326.22,3880.59,4434.96,4989.33,5543.7,6098.07,6652.44
5,PRODUCT MANAGER,1132.78,2265.56,3398.34,4531.12,5663.90,6796.68,7929.46,9062.24,10195.02,11327.8,12460.58,13593.36
6,WEB DESIGNER,2124.20,4248.40,6372.60,8496.80,10621.00,12745.20,14869.40,16993.60,19117.80,21242.0,23366.20,25490.40


## Same queries in pandas

In [93]:
job = pd.read_sql_query("SELECT * FROM job", con)
workers = pd.read_sql_query("SELECT * FROM workers", con)
con.close()

In [94]:
job.loc[(job["SALARY"] >= 400) & (job["SALARY"] <= 600), "POSITION"].unique()

array(['C# DEVELOPER', 'AUTOMATION QA ENGINEER', 'JAVA DEVELOPER'],
      dtype=object)

In [95]:
(job[job["OFFICE_CITY"].isin(["SEBEWAING", "ELECTRON"])]
.groupby("OFFICE")
.agg({"WORKER_ID": "count", "SALARY": "mean"}))

,WORKER_ID,SALARY
OFFICE,,
TIME IS MONEY,7,725.958571


In [96]:
(job
.groupby("POSITION")
.filter(lambda g: all((g["SALARY"] <= 700) & (g["SALARY"] >= 400)))["POSITION"]
.unique())

array(['AUTOMATION QA ENGINEER', 'JAVA DEVELOPER'], dtype=object)

In [97]:
workers_new = (workers
                .assign(AGE=(pd.to_datetime("today") -
                                pd.to_datetime(workers["DOB"], format="%d-%b-%y")) //
                                np.timedelta64(1, "Y")))

workers_merged = pd.merge(workers_new, job, on="WORKER_ID")

(workers_merged
.query("AGE > 30")
.groupby(["GENDER", "AGE"])
.agg(SALARY=pd.NamedAgg(column="SALARY", aggfunc=(lambda x: np.mean(x) * 1.2))))

SALARY
GENDER AGE          
F      33   1052.370
       34   1175.004
       38    665.244
M      33    691.544

In [98]:
d_range = (pd.Series(pd.date_range(start=min(pd.to_datetime(job["FROM_DATE"],
                                            format="%d-%b-%y")),
                                  end=pd.to_datetime("now"),
                                  freq="M"))
                    .to_frame(name="range")
                    .set_index("range"))


job_filtered = (job
                .loc[job["WORKER_ID"] == workers
                                        .loc[(workers["FIRST_NAME"] == "NIKOLA") &
                                            (workers["LAST_NAME"] == "TESLA"), "WORKER_ID"]
                                        .values[0],
                    ["SALARY", "FROM_DATE"]]
    .assign(FROM_DATE=pd.to_datetime(job["FROM_DATE"],
                                    format="%d-%b-%y"))
    .set_index("FROM_DATE"))


(pd.merge_asof(d_range,
              job_filtered,
              left_index=True,
              right_index=True)
.dropna(axis=0)
.cumsum())

,SALARY
range,
2009-06-30,621.44
2009-07-31,1242.88
2009-08-31,1864.32
2009-09-30,2485.76
2009-10-31,3107.20
...,...
2019-12-31,78922.88
2020-01-31,79544.32
2020-02-29,80165.76
